In [1]:
!pip install scikeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.8.2 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.

In [2]:
import pandas as pd
import tensorflow.keras.backend as k
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor

In [3]:
ds = pd.read_csv('/kaggle/input/card-12-games/games.csv')
ds = ds.sample(frac=0.5) # para tornar a grid search viável
ds

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
6467,Major League Baseball 2K10,Wii,2010.0,Sports,Take-Two Interactive,0.24,0.00,0.00,0.02,0.26,53.0,4.0,tbd,NaN,Visual Concepts,E
16673,Super Robot Taisen: Original Generation,GBA,2002.0,Role-Playing,Banpresto,0.01,0.00,0.00,0.00,0.01,72.0,13.0,tbd,NaN,Banpresto,T
4122,Up,PS2,2009.0,Action,THQ,0.19,0.05,0.00,0.24,0.48,NaN,NaN,tbd,NaN,Asobo Studio,E
3525,Wreckless: ThE YaKuza MisSiOns,XB,2002.0,Racing,Activision,0.42,0.11,0.02,0.02,0.57,74.0,36.0,5.2,13.0,Bunkasha Publishing,T
2544,NCAA Football 10,PS3,2009.0,Sports,Electronic Arts,0.75,0.00,0.00,0.06,0.81,80.0,20.0,6.9,14.0,EA Tiburon,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4792,Mass Effect Trilogy,X360,2012.0,Action,Electronic Arts,0.28,0.09,0.00,0.03,0.40,NaN,NaN,7.4,30.0,BioWare,M
2425,Dynasty Warriors 4: Xtreme Legends,PS2,2003.0,Action,Tecmo Koei,0.17,0.13,0.51,0.04,0.86,72.0,15.0,9.1,39.0,Omega Force,T
6358,Thoroughbred Breeder II,SNES,1994.0,Simulation,Hect,0.00,0.00,0.27,0.00,0.27,NaN,NaN,NaN,NaN,NaN,NaN
9915,Bishoujo Senshi Sailormoon S: Juugai Rantou!? ...,SNES,1994.0,Fighting,Angel Studios,0.00,0.00,0.12,0.00,0.12,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
toDrop = ['Name', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Developer']
for i in toDrop:
    ds = ds.drop(i, axis=1)
ds

,Platform,Year_of_Release,Genre,Publisher,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
6467,Wii,2010.0,Sports,Take-Two Interactive,0.26,53.0,4.0,tbd,NaN,E
16673,GBA,2002.0,Role-Playing,Banpresto,0.01,72.0,13.0,tbd,NaN,T
4122,PS2,2009.0,Action,THQ,0.48,NaN,NaN,tbd,NaN,E
3525,XB,2002.0,Racing,Activision,0.57,74.0,36.0,5.2,13.0,T
2544,PS3,2009.0,Sports,Electronic Arts,0.81,80.0,20.0,6.9,14.0,E
...,...,...,...,...,...,...,...,...,...,...
4792,X360,2012.0,Action,Electronic Arts,0.40,NaN,NaN,7.4,30.0,M
2425,PS2,2003.0,Action,Tecmo Koei,0.86,72.0,15.0,9.1,39.0,T
6358,SNES,1994.0,Simulation,Hect,0.27,NaN,NaN,NaN,NaN,NaN
9915,SNES,1994.0,Fighting,Angel Studios,0.12,NaN,NaN,NaN,NaN,NaN


In [5]:
ds.isnull().sum()

Platform              0
Year_of_Release     141
Genre                 1
Publisher            24
Global_Sales          0
Critic_Score       4335
Critic_Count       4335
User_Score         3420
User_Count         4582
Rating             3457
dtype: int64

In [6]:
catCols = ['Year_of_Release', 'Genre', 'Publisher', 'Rating']
for i in catCols:
    mode_val = ds[i].mode()[0]
    ds[i] = ds[i].fillna(mode_val)
ds.isnull().sum()

Platform              0
Year_of_Release       0
Genre                 0
Publisher             0
Global_Sales          0
Critic_Score       4335
Critic_Count       4335
User_Score         3420
User_Count         4582
Rating                0
dtype: int64

In [7]:
ds.loc[ds['User_Score'] == 'tbd', 'User_Score'] = None
ds['User_Score'] = ds['User_Score'].astype(float)

In [8]:
numCols = ['Critic_Score', 'Critic_Count', 'User_Score', 'User_Count']
for i in numCols:
    ds[i] = ds[i].fillna(ds[i].mean())

In [9]:
ds.isnull().sum()

Platform           0
Year_of_Release    0
Genre              0
Publisher          0
Global_Sales       0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
Rating             0
dtype: int64

In [10]:
X = ds.iloc[:, [0, 1, 2, 3, 5, 6, 7, 8, 9]].values
y = ds.iloc[:, 4].values

In [11]:
toOneHotCols = [0, 2, 3, 8]
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), toOneHotCols)], remainder = 'passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [12]:
X.shape

(8360, 502)

In [13]:
inputNeurons = X.shape[1]
def create_network(dropout):
    k.clear_session()
    input_layer = Input(shape=(inputNeurons,))
    hidden_layer1 = Dense(units = int(inputNeurons/2))(input_layer)
    activation_layer1 = Activation('relu')(hidden_layer1)
    dropout_layer1 = Dropout(rate = dropout)(activation_layer1)
    hidden_layer2 = Dense(units = int(inputNeurons/2))(dropout_layer1)
    activation_layer2 = Activation('relu')(hidden_layer2)
    dropout_layer2 = Dropout(rate = dropout)(activation_layer2)
    output_layer = Dense(units = 1, activation = 'linear')(dropout_layer2)
    
    regressor = Model(inputs = input_layer, outputs = output_layer)
    regressor.compile(optimizer = 'adam', loss = 'mse')
    
    return regressor

In [14]:
params = {
    'batch_size': [100, 200],
    'epochs': [500, 1000],
    'model__dropout': [0.2, 0.3]
}

In [15]:
regressor = KerasRegressor(model = create_network)

In [16]:
grid_search = GridSearchCV(estimator = regressor, param_grid = params,
                          cv = 3, scoring='neg_mean_squared_error')

In [17]:
grid_search = grid_search.fit(X, y)

Epoch 1/500


I0000 00:00:1725027020.673832      71 service.cc:145] XLA service 0x7f1a68002e50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725027020.673903      71 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


33/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2505.9524 

I0000 00:00:1725027023.841217      71 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


56/56 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - loss: 2054.4661
Epoch 2/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 193.2381
Epoch 3/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 95.2561 
Epoch 4/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 52.4475
Epoch 5/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 49.8128
Epoch 6/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15.8663
Epoch 7/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10.0526
Epoch 8/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10.5537
Epoch 9/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.2864
Epoch 10/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.2920
Epoch 11/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.3368
Epoch 12/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.5366
Epoch 13/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4.1746
Epoch 14/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.3802
Epoch 15/500
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.6078
Epo

In [18]:
grid_search.best_params_

{'batch_size': 200, 'epochs': 1000, 'model__dropout': 0.2}

In [19]:
grid_search.best_score_

-1.384059324779951